# Crash course in Scheme

To follow along, make sure you have Calysto scheme installed

```
pip install calysto-shceme
```

Scheme is a dialect of LISP. We're teaching basic Scheme to get acquainted with some basic concepts that become important later on:

* The syntax tree is explicit: S-expressions are both data and code.
* Everything is an expression, there are *no statements* in Scheme.
* Scheme eliminates tail-calls, loops are done through recursion.

There are many more interesting parts of Scheme, like first-class continuations and hygienic macros, but these are not important for our discussion on parallel programming in Python.

If you are interested though, here are some good references:

* ["Structure and Interpretation of Computer Programs" by Harold Abelson and Gerald Sussman](https://mitpress.mit.edu/sites/default/files/sicp/index.html), a classic that should be on anyones book shelves.
* ["The Scheme Programming Language" by R. Kent Dybvig](https://www.scheme.com/tspl4/), a good complete reference.
* The Little Schemer, The Seasoned Schemer, The Reasoned Schemer, The Little Typer series 
* [https://schemers.org/](https://schemers.org/)

and implementations (faster and better than Calysto scheme, which is hosted on Python):

* [Chez Scheme](https://cisco.github.io/ChezScheme/), also by Kent Dybvig, fastest Scheme compiler out there.
* [Racket](https://racket-lang.org/), not just Scheme but an ecosystem of languages with a big community.
* [Guile](http://gnu.org/s/guile/), GNU's implementation of Scheme.

## Syntax: S-expressions

Scheme expressions can be booleans, numbers, strings, symbols and lists.

In [1]:
#t  ; true

#t

In [2]:
42   ; a number

42

In [3]:
"Hello, World!"  ; a string

"Hello, World!"

In [4]:
'x  ; a symbol

x

In [5]:
'(a b c)  ; a list of symbols

(a b c)

In [6]:
'(42 is the answer)  ; a list

(42 is the answer)

Note that for the symbol and the list, we had to use a single quote mark before the expression. We'll get to quoting in a moment. For now, try these without the quotes and see if you understand what happens.

### Function application

Let's look at some functions in Scheme. Functions are applied by entering a list where the first element is the function, and the tail of the list gives the arguments.

In [7]:
(+ 1 2)

3

In [8]:
(string-length "Hello, World!")

13

In [9]:
(substring "Hello, World!" 0 5)

"Hello"

### Quoting

Since a list or a symbol is normally interpreted, we need a way to denote a list that is just data:

In [10]:
(list 1 2 3)

(1 2 3)

In [11]:
(quote (1 2 3))

(1 2 3)

In [12]:
'(1 2 3)

(1 2 3)

or a symbol that is just a symbol

In [13]:
(quote symbol)

symbol

In [14]:
'symbol

symbol

### Functions

Functions are constructed using the `lambda` keyword

In [15]:
((lambda (x) (* x 5)) 10)

50

In [16]:
((lambda (name) (string-append "Hello, " name "!")) "Felipe")

"Hello, Felipe!"

Functions are not much use if we can't give them a name.

### `let` bindings

In [17]:
(let ((alice "Alice")
      (bob   "Bob")
      (greet (lambda (name)
               (display (string-append "Hello, " name "!"))
               (newline)
               123)))
  (greet alice)
  (greet bob))

Hello, Alice!
Hello, Bob!


123

> Exercise: `let` is actually a macro that expands to an expression involving a `lambda`, can you think what this would look like? How would you write `(let ((a 2) (b 3)) (+ a b))` in a lambda expression preserving the `(+ a b)` body?

> Answer: `(let ((a 2) (b 3)) (+ a b))` expands to `((lambda (a b) (+ a b)) 2 3)`.

### `define`

In [57]:
(define five 5)

(define square (lambda (x) (* x x)))

(display (square 8)) (newline)
(display (square x)) (newline)

64
25


For defining functions there is special syntax.

```scheme
(define (square x) (* x x))
```

is identical to

```scheme
(define square (lambda (x) (* x x)))
```

In [58]:
(define (greet name)
  (display (string-append "Hello, " name "!\n")))

In [59]:
(greet "uncle")

Hello, uncle!


### `if` expressions

In [20]:
(if #f
    1
    2)

2

## Recursion

We have not seen any looping constructs yet! That's because Scheme does not have loops. To illustrate things, we define a new number system.

In [45]:
(define zilch "")

(define unit "I")

(define (inc a)
  (string-append a unit))

(define (dec a)
  (substring a 1))

(define (zilch? a) (string=? a zilch))

Using this number system, let's define addition.

In [46]:
(define (add a b)
  (if (zilch? a)
      b
      (add (dec a) (inc b))))

In [47]:
(add "III" "II")

"IIIII"

Can you define multiplication? (hint: first define the `unit?` function)

In [48]:
(define (unit? a) (string=? a unit))

(define (mul a b)
  (if (unit? a)
      b
      (add (mul (dec a) b) b)))

In [49]:
(mul "III" "IIIII")

"IIIIIIIIIIIIIII"

There is a difference in which the `add` and `mul` functions recurse. In the case of `add`, the recursive call happens in the *tail*. Suppose you have to derive by hand how `(add "III" "II")` works by substituting a function call for the value it returns:

* `(add "III" "II")`
* `(if (zilch? "III") "II" (add (dec "III") (inc "II")))`
* `(add (dec "III") (inc "II"))`
* `(add "II" "III")`
* ... skip to tail ...
* `(add "I" "IIII")`
* ... skip to tail ...
* `(add "" "IIIII")`
* ... skip to tail ...
* `"IIIII"`

Every recursion, the original call is being *replaced* by a new call. Now we do the same for `(mul "III" "IIIII")`.

* `(mul "III" "IIIII")`
* `(add (mul "II" "IIIII") "IIIII")`
* `(add (add (mul "I" "IIIII") "IIIII") "IIIII")`
* `(add (add "IIIII" "IIIII") "IIIII")`
* `(add "IIIIIIIIII" "IIIII")`
* `"IIIIIIIIIIIIIII"`

The `mul` function is not *tail-recursive*. 

In [50]:
(define (madd a b c)
  (if (zilch? a)
      c
      (madd (dec a) b (add b c))))

(define (mul a b) (madd a b zilch))

Tracing the `(mul "III" "IIIII")` call now results in

* `(mul "III" "IIIII")`
* `(madd "III" "IIIII" "")`
* `(madd "II" "IIIII" "IIIII")`
* `(madd "I" "IIIII" "IIIIIIIIII")`
* `(madd "" "IIIII" "IIIIIIIIIIIIIII")`
* `"IIIIIIIIIIIIIII"`

> Exercise: write a tail-recursive function that computes the factorial (of a normal number).

> Hints: there is a `zero?` function that checks if a number equals 0.

# Eval

Scheme is a **homoiconic** language. This means that code looks like data and code can be trivially stored as such. We can quote a piece of code, pass it around, or modify it for later evaluation.

In [71]:
(let* ((expr1 '(+ 6 7))
       (expr2 (cons '* (cdr expr1))))
  (display expr1) (newline)
  (display " => ") (display (eval expr1)) (newline)
  (display expr2) (newline)
  (display " => ") (display (eval expr2)) (newline))

(+ 6 7)
 => 13
(* 6 7)
 => 42
